# Impact du preprocessing 

## Rappels 

**Le preprocessing est une étape cruciale dans le pipeline de machine learning.**  
Il permet de préparer les données pour les modèles, en assurant que les caractéristiques sont sur une échelle comparable et que les données sont propres et prêtes à être utilisées.

**Certains modèles sont plus impactés par le preprocessing que d'autres.**  
Par exemple, les modèles basés sur des distances (comme les k-NN ou les SVM) sont très sensibles à l'échelle des données, tandis que les arbres de décision ne le sont pas autant.

### Règle mentale essentielle - Scaler :
> **Si le modèle calcule une distance, un produit scalaire ou optimise par gradient → SCALE.**   
> **S’il coupe avec des seuils (arbres) → pas besoin.**  

#### Le modèle compare des points ? 
(distance, similarité, projection) ➡ Scaler obligatoire

Exemples :
- KNN

- K-Means (clustering)

- SVM

- Régression logistique / linéaire

- Réseaux de neurones (deep learning)

- PCA

#### Le modèle fait des "si X < seuil" ?

(découpe en branches) ➡ Pas besoin de scaling

Exemples :

- Decision Tree

- Random Forest

- Gradient Boosting

### Règle mentale - Encoding : 
> **si le modèle comprend une notion d’ordre → l’ordinal peut aller**  
> **Sinon → One-Hot Encoding.**  

#### Le modèle utilise des distances ou des produits scalaires ?

➡ One-Hot Encoding obligatoire.  
➡ Pas d’encoding ordinal arbitraire. 

Exemples :
- KNN

- SVM

- Régression logistique / linéaire

- MLP

- K-Means

**Pourquoi ?**

Si tu codes :
```sh
Chat = 0
Chien = 1
Lion = 2
```
➡ Le modèle croit que Lion > Chien > Chat, ce qui est faux.

#### Le modèle est basé sur des arbres ?

➡ Peut tolérer :

- Ordinal Encoding

- Label Encoding

- (One-Hot reste sûr)

Exemples :

- Decision Tree

- Random Forest

- Gradient Boosting

**Pourquoi ?**

Les arbres coupent avec des seuils :
```sh
si catégorie <= 1
```
➡ Ils ne calculent pas de distance.

---

## Exemples d'impact du preprocessing

In [1]:
import numpy as np
from sklearn.datasets import load_wine, fetch_openml
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline # pour faire du chaining de transformations + modèle
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer # pour faire du preprocessing différent sur différentes colonnes
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [2]:
# --------- 1) SCALING impact (réel, sklearn) : WINE ----------
Xw, yw = load_wine(return_X_y=True)

models_scaling = {
    "KNN sans scaling": KNeighborsClassifier(5),
    "KNN + scaling": make_pipeline(StandardScaler(), KNeighborsClassifier(5)),
    "SVM RBF sans scaling": SVC(),
    "SVM RBF + scaling": make_pipeline(StandardScaler(), SVC()),
}
print("\n=== SCALING (Wine) : accuracy CV=5 ===")
for name, m in models_scaling.items():
    acc = cross_val_score(m, Xw, yw, cv=5, scoring="accuracy").mean()
    print(f"{name:20s} -> {acc:.3f}")


=== SCALING (Wine) : accuracy CV=5 ===
KNN sans scaling     -> 0.691
KNN + scaling        -> 0.949
SVM RBF sans scaling -> 0.663
SVM RBF + scaling    -> 0.983


In [ ]:
# --------- 2) ENCODING impact (réel via OpenML) : TITANIC ----------
print("\n=== ENCODING (Titanic) : accuracy CV=5 ===")
try:
    titanic = fetch_openml("titanic", version=1, as_frame=True)
    print("Données Titanic téléchargées depuis OpenML.")
    
    X, y = titanic.data, titanic.target
    y = (y == "1").astype(int)  # survécu=1

    cat = X.select_dtypes(include=["object", "category", "string"]).columns
    num = X.columns.difference(cat)

    # Creation des pipelines de preprocessing
    num_pipe = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())
    
    # Ordinal (souvent mauvais pour modèles à distance/linéaires)
    cat_pipe_ord = make_pipeline(SimpleImputer(strategy="most_frequent"),
                              OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
    
    pre_ord = ColumnTransformer([
        ("num", num_pipe, num),
        ("cat", cat_pipe_ord, cat)
    ])
    
    # One-Hot (recommandé)
    cat_pipe_ohe = make_pipeline(SimpleImputer(strategy="most_frequent"),
                              OneHotEncoder(handle_unknown="ignore"))
    
    pre_ohe = ColumnTransformer([
        ("num", num_pipe, num),
        ("cat", cat_pipe_ohe, cat)
    ])

    knn_ord = make_pipeline(pre_ord, KNeighborsClassifier(7))
    knn_ohe = make_pipeline(pre_ohe, KNeighborsClassifier(7))
    log_ord = make_pipeline(pre_ord, LogisticRegression(solver="saga", max_iter=10000)) # solver "saga" gère les données dispersées (sparse) et est rapide pour les grands datasets
    log_ohe = make_pipeline(pre_ohe, LogisticRegression(solver="saga", max_iter=10000))
    
    models_encoding = {
        "KNN + OrdinalEnc": knn_ord,
        "KNN + OneHotEnc": knn_ohe,
        "LogReg + OrdinalEnc": log_ord,
        "LogReg + OneHotEnc": log_ohe,
    }

    for name, m in models_encoding.items():
        acc = cross_val_score(m, X, y, cv=5, scoring="accuracy").mean()
        print(f"{name:20s} -> {acc:.3f}")

except Exception as e:
    print("Impossible de télécharger Titanic via OpenML (hors-ligne ?).")
    print("Erreur:", type(e).__name__, "-", str(e)[:120])
    print("Astuce: remplace Titanic par un CSV local (Kaggle) et réutilise les mêmes pipelines.")


=== ENCODING (Titanic) : accuracy CV=5 ===
Données Titanic téléchargées depuis OpenML.
KNN + OrdinalEnc     -> 0.488
KNN + OneHotEnc      -> 0.791
LogReg + OrdinalEnc  -> 0.658
LogReg + OneHotEnc   -> 0.959
